In [1]:
import pandas as pd
import altair as alt
from altair_saver import save
import numpy as np
from IPython.display import Image
pd.options.mode.chained_assignment = None  # default='warn'

# Ranking comparison - surverys and dueling-bandints

## Read in data

In [2]:
df_free = pd.read_csv('../clean_data_mturk/free_choice_emotion_mturk.csv')
df_free_labels = pd.read_csv('../data_mturk/emotion_labels_free_choice.csv')

df_forced = pd.read_csv('../clean_data_mturk/forced_choice_emotion_mturk.csv')
df_forced_labels = pd.read_csv('../data_mturk/emotion_labels.csv')

In [3]:
df_emo_answers_free = df_free.loc[:, 'Q2.1_1':'Q195.1_4'] # get cols with words only
df_emo_answers_forced = df_forced.loc[:, 'Q2.1':'Q195.1'] 

## Text pre-processing

In [4]:
# word clean up
def manual_spell_check(df):
    """manual replacement of mispelled words"""
    
    df.replace(['happy', 'happiness', 'happyness', 'hapiness', 'happines', 'happpy',\
                                  'happyniess', 'happyu', 'hapy', 'happynes','hapyness'], 'happiness', inplace=True)
    
    df.replace(['angry', 'anger', 'angery', 'angered', 'angary', 'angri'], 'anger', inplace=True)
    
    df.replace(['sad', 'sandness', 'sadnesss', 'sadness', 'sading', 'sand'], 'sadness', inplace=True)
    
    df.replace(['disgust', 'disgusted', 'disgusting', 'disgus', 'disgusting'], 'disgust', inplace=True)
    
    df.replace(['surprised', 'surprise', 'surprising', 'surpise'], 'surprise', inplace=True)
    
    df.replace(['fear', 'fearful', 'fearsome', 'fera'], 'fear', inplace=True)
    
    df.replace(['nerural', 'neutral', 'netural', 'netrul', 'neutal'], 'neutral', inplace=True)
    
    return df

In [5]:
# df_emo_answers_free = manual_replace(df_emo_answers_free)

In [6]:
def emotion_df_formated(df_emo_answers, emotion_label, df_labels):
    df_emo_cat = df_emo_answers.copy() 
    df_emo_cat_t = df_emo_cat.T # transpose
    df_emo_cat_t['photo_id'] = df_emo_cat_t.index # get index as col
    df_emo_cat_t = df_emo_cat_t.reset_index(drop=True) # clean index
    df_emo_cat_t_labels = pd.concat([df_emo_cat_t, df_labels], axis=1) # add metadata cols
    df_label =  df_emo_cat_t_labels[df_emo_cat_t_labels['label'] == emotion_label]
    
    return df_label

In [7]:
def formating_words(df, len_words=1, len_letters=2):
    df_stack = df.stack().reset_index(drop=True) # stack as series
    df_stack = df_stack.to_frame(name='emotion') # as DF
    df_stack['emotion'] = df_stack['emotion'].str.strip() # remove blank spaces
    df_stack['emotion'] = df_stack['emotion'].str.lower() # as lower case
    df_stack['emotion'] = df_stack['emotion'].replace({'na':'none'}) 
    df_stack['len_words'] = df_stack['emotion'].str.split().apply(len) # cnt number of words
    df_stack['len_letters'] = df_stack['emotion'].apply(len) # cont number of letters
    # get df with single words of 3 or more letters
    df_stack_single_word = df_stack[(df_stack['len_words'] == len_words) & (df_stack['len_letters'] > len_letters)] 
    df_stack_single_word['emotion'] = df_stack_single_word['emotion'].str.replace('[^a-zA-Z]', '') # remove non-alphabetic characters
    df_stack_single_word = df_stack_single_word[~df_stack_single_word['emotion'].str.contains(r'[0-9]')] #drop words containing numbers
    df_stack_single_word = df_stack_single_word[df_stack_single_word.groupby('emotion').emotion.transform(len) > 1] # keep words that ocurr > 1
    return df_stack_single_word

## Grouping 

In [8]:
# importing the module
import json
 
# synsets 
with open('../clean_data/syn_dict_emotions.json') as json_file:
    syns = json.load(json_file)

# hyponyms 
with open('../clean_data/hyp_dict_emotions.json') as json_file:
    hyps = json.load(json_file)

In [9]:
def update_dic(syns, emo, emo_variant):
    '''to merge key variants'''
    
    syns[emo].extend(syns[emo_variant])
    syns.pop(emo_variant)

In [10]:
update_dic(syns, emo='anger', emo_variant='angry')
update_dic(syns, emo='disgust', emo_variant='disgusted')
update_dic(syns, emo='sadness', emo_variant='sad')
update_dic(syns, emo='fear', emo_variant='fearful')
update_dic(syns, emo='surprise', emo_variant='surprised')
update_dic(syns, emo='happiness', emo_variant='happy')

In [11]:
def grouping(df, syns):
    '''to group (replace) by synsets and hyponyms'''
    
    for key in syns:
        synset = syns[key]
        df.replace(synset, key, inplace=True)
    
    return df

## Chart functions

In [12]:
def simple_per_bar_concat(
    df, title='Title', X='percent:Q', Y='emotion:N', \
    width=450, height=250, sort='-x', \
    text_size = 12, label_size = 11, title_size=12, \
    emotion='Some', color1='#0570b0', color2='orange'):
    
    bars = alt.Chart(df, title=title).mark_bar().encode(
        alt.X(X, axis=alt.Axis(format='.0%')),
        y=alt.Y(Y, sort=sort), 
        color=alt.condition(
            alt.datum.emotion == emotion,
            alt.value(color2),
            alt.value(color1)
        ))

    text_placeholder = alt.Chart(df).mark_circle(color="black", opacity=0.1).encode(
        alt.X(X),
        alt.Y(Y, sort=sort))

    text = text_placeholder.mark_text(
    align='left',
    baseline='middle',
    dx=3,  # Nudges text to right so it doesn't appear on top of the bar
    fontSize=text_size
    ).encode(
        alt.Text(X, format='.1%')
    )
    
    chart = (bars + text)
    
    return chart

In [13]:
def simple_cnt_bar_concat(
    df, title='Title', X='percent:Q', Y='emotion:N', \
    width=450, height=250, sort='-x', \
    text_size = 12, label_size = 11, title_size=12, \
    emotion='Some', color1='#0570b0', color2='orange'):
    
    bars = alt.Chart(df, title=title).mark_bar().encode(
        alt.X(X, axis=alt.Axis(format='.0',title='Borda score(bar and number), Score precision (horizontal line)')),
        y=alt.Y(Y, sort=sort), 
        color=alt.condition(
            alt.datum.emotion == emotion,
            alt.value(color2),
            alt.value(color1)
        ))
    
    rule =alt.Chart(df).mark_rule(color="black").encode(
    alt.X('lower:Q', scale=alt.Scale(zero=False)),
    alt.X2('upper:Q'),
    alt.Y(Y,sort=alt.SortField(field="borda score", order='descending')))

    text_placeholder = alt.Chart(df).mark_circle(opacity=0.1).encode(
        alt.X('upper:Q'),
        alt.Y(Y,sort=alt.SortField(field="borda score", order='descending')))

    text = text_placeholder.mark_text(
    align='left',
    baseline='middle',
    dx=3,  # Nudges text to right so it doesn't appear on top of the bar
    fontSize=text_size
    ).encode(
        alt.Text(X)
    )
    
    chart = (bars + text + rule)
    
    return chart

In [14]:
def count_freq_labels(df, X="all", col="emotion"):
    if X == "all":
        df_counts = df.stack().reset_index(drop=True).value_counts() # stack as series
        df_counts = df_counts.to_frame('counts') # get value_counts as df
        df_counts['emotion'] = df_counts.index # get index as col
    elif X == "stacked":
        df_counts = df[col].value_counts() # count word frequency
        df_counts = df_counts.to_frame('counts') # get value_counts as df
        df_counts['emotion'] = df_counts.index # get index as col    
    else:
        df_counts = df[X].reset_index(drop=True).value_counts() # stack as series
        df_counts = df_counts.to_frame('counts') # get value_counts as df
        df_counts[X] = df_counts.index # get index as col

    df_counts = df_counts.reset_index(drop=True) # clean index
    df_counts['percent'] = df_counts['counts'] / df_counts['counts'].sum() # compute percentage
    return df_counts

In [15]:
def photo_chart(df_emo_answers, emotion,  i, df_labels, yoffset=220, emotion_st = None, n_mentions=2, photo = None):
    df = emotion_df_formated(df_emo_answers, emotion, df_labels) # add emotion label
    photo_id_list = df['photo_id'].tolist() # photo id to list
    splited_photo_id = np.array_split(photo_id_list, 24) # split photo id by photo
    df = df[df['photo_id'].isin(splited_photo_id[i])] # select photo rows   
    df_url = df[['url']].head(1)  # get url for chart
    
    face = alt.Chart(df_url).mark_image(width=150, height=150, align='center', xOffset=-100, yOffset=yoffset).encode(url='url')
    
    df_emotion_ans = df.drop(['photo_id', 'ethnicity', 'sex', 'age', 'label', 'url', 'photoId'], axis=1) # clean cols
    df_stack_emotion = formating_words(df_emotion_ans) # clean up words
    #df_stack_emotion['emotion_ps_steamed'] = df_stack_emotion['emotion'].apply(lambda x: ps.stem(x)) # steam
    df_stack_emotion = manual_spell_check(df_stack_emotion) # manual fix
    df_stack_emotion = grouping(df_stack_emotion, syns) # group by synonyms
    df_stack_emotion = grouping(df_stack_emotion, hyps) # group by hyponyms
    df_stack_emotion = df_stack_emotion.replace({'scared': 'fear'}) # manual scare fix
    df_stack_emotion_count = count_freq_labels(df_stack_emotion, X="stacked", col='emotion') # group and count
    source = df_stack_emotion_count[df_stack_emotion_count['counts'] >= n_mentions]
    
    chart = simple_per_bar_concat(source,\
                           title=f"Categories with {n_mentions} or more mentions | n= {source['counts'].sum().astype(str)}", \
                           X='percent:Q', Y='emotion:N', emotion=emotion_st)

    if photo == True:
        return chart + face
    else:
        return chart

In [16]:
def next_chart_wrapper(file_path, emotion, head):
    df = pd.read_csv(file_path)
    
    df = df.round(decimals=2)
    
    df = df.replace({'Target' : {'angry' : 'anger', 
                                 'disgusted' : 'disgust',
                                 'fearful' : 'fear', 
                                 'happy': 'happiness', 
                                 'surprised': 'surprise',
                                  'sad': 'sadness'}})

    df['upper'] = df['Score'] + df['Precision'] /2
    df['lower'] = df['Score'] - df['Precision'] /2
    
    df.rename(columns={
        'Target': 'emotion',
        'Score': 'borda score'}, inplace=True)

    title=f"Dueling bandits ranking | Top: {head}"

    chart = simple_cnt_bar_concat(df.head(head), title= title, \
                                  X='borda score:Q', Y='emotion:N', emotion=emotion)
    return chart

## Anger - Female of color

In [17]:
anger_bfa_forced = photo_chart(df_emo_answers_forced, 'anger',  0, df_forced_labels, emotion_st='anger', n_mentions=2, photo=True, yoffset=280)
anger_bfa_free = photo_chart(df_emo_answers_free, 'anger',  0, df_free_labels, emotion_st='anger', n_mentions=2)

/tmp/ipykernel_11424/2691402293.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_stack_single_word['emotion'] = df_stack_single_word['emotion'].str.replace('[^a-zA-Z]', '') # remove non-alphabetic characters


In [18]:
path = '../../emo-ranking-next/anger_bipoc_female/mturk/borda_lilucb_ranking_mturk.csv'

anger_bfa_next = next_chart_wrapper(path, emotion='anger', head=54)

In [19]:
anger_bfa_next_panel = (anger_bfa_forced.properties(width=200) | anger_bfa_free.properties(width=200) | anger_bfa_next.properties(width=200)).properties(title="Female of color - Anger")
anger_bfa_next_panel.configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0).configure_title(dy=-12)

alt.HConcatChart(...)

In [20]:
save(anger_bfa_next_panel, "../../emotions_dashboard/data/anger_bfa_next_panel_mturk.png")

## Anger - Male of color

In [21]:
anger_bma_forced = photo_chart(df_emo_answers_forced, 'anger',  2, df_forced_labels, emotion_st='anger', n_mentions=2, photo=True)
anger_bma_free = photo_chart(df_emo_answers_free, 'anger',  2, df_free_labels, emotion_st='anger', n_mentions=2)

/tmp/ipykernel_11424/2691402293.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_stack_single_word['emotion'] = df_stack_single_word['emotion'].str.replace('[^a-zA-Z]', '') # remove non-alphabetic characters


In [22]:
path = '../../emo-ranking-next/anger_bipoc_male/mturk/borda_lilucb_ranking_mturk.csv'

anger_bma_next = next_chart_wrapper(path, emotion='anger', head=35)

In [23]:
anger_bma_next_panel = (anger_bma_forced.properties(width=200) | anger_bma_free.properties(width=200) | anger_bma_next.properties(width=200)).properties(title="Male of color - Anger")
anger_bma_next_panel.configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0).configure_title(dy=-12)

alt.HConcatChart(...)

In [24]:
save(anger_bma_next_panel, "../../emotions_dashboard/data/anger_bma_next_panel_mturk.png")

## Anger - White Female 

In [25]:
anger_wfa_forced = photo_chart(df_emo_answers_forced, 'anger',  12, df_forced_labels, emotion_st='anger', n_mentions=2, photo=True)
anger_wfa_free = photo_chart(df_emo_answers_free, 'anger',  12, df_free_labels, emotion_st='anger', n_mentions=2)

/tmp/ipykernel_11424/2691402293.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_stack_single_word['emotion'] = df_stack_single_word['emotion'].str.replace('[^a-zA-Z]', '') # remove non-alphabetic characters


In [26]:
path = '../../emo-ranking-next/anger_white_female/mturk/borda_lilucb_ranking_mturk.csv'

anger_wfa_next = next_chart_wrapper(path, emotion='anger', head=10)

In [27]:
anger_wfa_next_panel = (anger_wfa_forced.properties(width=200) | anger_wfa_free.properties(width=200) | anger_wfa_next.properties(width=200)).properties(title="White female - Anger")
anger_wfa_next_panel.configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0).configure_title(dy=-12)

alt.HConcatChart(...)

In [28]:
save(anger_wfa_next_panel, "../../emotions_dashboard/data/anger_wfa_next_panel_mturk.png")

## Anger - White male

In [29]:
anger_wma_forced = photo_chart(df_emo_answers_forced, 'anger',  6, df_forced_labels, emotion_st='anger', n_mentions=2, photo=True, yoffset=260)
anger_wma_free = photo_chart(df_emo_answers_free, 'anger',  6, df_free_labels, emotion_st='anger', n_mentions=2)

/tmp/ipykernel_11424/2691402293.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_stack_single_word['emotion'] = df_stack_single_word['emotion'].str.replace('[^a-zA-Z]', '') # remove non-alphabetic characters


In [30]:
path = '../../emo-ranking-next/anger_white_male/mturk/borda_lilucb_ranking_mturk.csv'

anger_wma_next = next_chart_wrapper(path, emotion='anger', head=15)

In [31]:
anger_wma_next_panel = (anger_wma_forced.properties(width=200) | anger_wma_free.properties(width=200) | anger_wma_next.properties(width=200)).properties(title="White male - Anger")
anger_wma_next_panel.configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0).configure_title(dy=-12)

alt.HConcatChart(...)

In [32]:
save(anger_wma_next_panel, "../../emotions_dashboard/data/anger_wma_next_panel_mturk.png")

In [33]:
## JOIN PANELS
anger_panel = (anger_bfa_next_panel & anger_bma_next_panel & anger_wfa_next_panel & anger_wma_next_panel).configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0)
save(anger_panel, "/mnt/c/Users/pablo/Desktop/dissertation-docs/images/anger_panel_mturk.png")

## Disgust - Female of color

In [34]:
disgust_bfa_forced = photo_chart(df_emo_answers_forced, 'disgust',  0, df_forced_labels, emotion_st='disgust', n_mentions=2, photo=True, yoffset=260)
disgust_bfa_free = photo_chart(df_emo_answers_free, 'disgust',  0, df_free_labels, emotion_st='disgust', n_mentions=2)

/tmp/ipykernel_11424/2691402293.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_stack_single_word['emotion'] = df_stack_single_word['emotion'].str.replace('[^a-zA-Z]', '') # remove non-alphabetic characters


In [35]:
path = '../../emo-ranking-next/disgust_bipoc_female/mturk/borda_lilucb_ranking_mturk.csv'

disgust_bfa_next = next_chart_wrapper(path, emotion='disgust', head=10)

In [36]:
disgust_bfa_next_panel = (disgust_bfa_forced.properties(width=200) | disgust_bfa_free.properties(width=200) | disgust_bfa_next.properties(width=200)).properties(title="Female of color - Disgust")
disgust_bfa_next_panel.configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0).configure_title(dy=-12)

alt.HConcatChart(...)

In [37]:
save(disgust_bfa_next_panel, "../../emotions_dashboard/data/disgust_bfa_next_panel_mturk.png")

## Disgust - Male of color

In [38]:
disgust_bma_forced = photo_chart(df_emo_answers_forced, 'disgust',  2, df_forced_labels, emotion_st='disgust', n_mentions=2, photo=True)
disgust_bma_free = photo_chart(df_emo_answers_free, 'disgust',  2, df_free_labels, emotion_st='disgust', n_mentions=2)

/tmp/ipykernel_11424/2691402293.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_stack_single_word['emotion'] = df_stack_single_word['emotion'].str.replace('[^a-zA-Z]', '') # remove non-alphabetic characters


In [39]:
path = '../../emo-ranking-next/disgust_bipoc_male/mturk/borda_lilucb_ranking_mturk.csv'

disgust_bma_next = next_chart_wrapper(path, emotion='disgust', head=10)

In [40]:
disgust_bma_next_panel = (disgust_bma_forced.properties(width=200) | disgust_bma_free.properties(width=200) | disgust_bma_next.properties(width=200)).properties(title="Male of color - Disgust")
disgust_bma_next_panel.configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0).configure_title(dy=-12)

alt.HConcatChart(...)

In [41]:
save(disgust_bma_next_panel, "../../emotions_dashboard/data/disgust_bma_next_panel_mturk.png")

## Disgust - White female

In [42]:
disgust_wfa_forced = photo_chart(df_emo_answers_forced, 'disgust',  4, df_forced_labels, emotion_st='disgust', n_mentions=2, photo=True, yoffset=240)
disgust_wfa_free = photo_chart(df_emo_answers_free, 'disgust',  4, df_free_labels, emotion_st='disgust', n_mentions=2)

/tmp/ipykernel_11424/2691402293.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_stack_single_word['emotion'] = df_stack_single_word['emotion'].str.replace('[^a-zA-Z]', '') # remove non-alphabetic characters


In [43]:
path = '../../emo-ranking-next/disgust_white_female/mturk/borda_lilucb_ranking_mturk.csv'

disgust_wfa_next = next_chart_wrapper(path, emotion='disgust', head=10)

In [44]:
disgust_wfa_next_panel = (disgust_wfa_forced.properties(width=200) | disgust_wfa_free.properties(width=200) | disgust_wfa_next.properties(width=200)).properties(title="White female - Disgust")
disgust_wfa_next_panel.configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0).configure_title(dy=-12)

alt.HConcatChart(...)

In [45]:
save(disgust_wfa_next_panel, "../../emotions_dashboard/data/disgust_wfa_next_panel_mturk.png")

## Disgust - White male

In [46]:
disgust_wma_forced = photo_chart(df_emo_answers_forced, 'disgust',  6, df_forced_labels, emotion_st='disgust', n_mentions=2, photo=True)
disgust_wma_free = photo_chart(df_emo_answers_free, 'disgust',  6, df_free_labels, emotion_st='disgust', n_mentions=2)

/tmp/ipykernel_11424/2691402293.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_stack_single_word['emotion'] = df_stack_single_word['emotion'].str.replace('[^a-zA-Z]', '') # remove non-alphabetic characters


In [47]:
path = '../../emo-ranking-next/disgust_white_male/mturk/borda_lilucb_ranking_mturk.csv'

disgust_wma_next = next_chart_wrapper(path, emotion='disgust', head=25)

In [48]:
disgust_wma_next_panel = (disgust_wma_forced.properties(width=200) | disgust_wma_free.properties(width=200) | disgust_wma_next.properties(width=200)).properties(title="White male - Disgust")
disgust_wma_next_panel.configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0).configure_title(dy=-12)

alt.HConcatChart(...)

In [49]:
save(disgust_wma_next_panel, "../../emotions_dashboard/data/disgust_wma_next_panel_mturk.png")

In [50]:
## JOIN PANELS
disgust_panel = (disgust_bfa_next_panel & disgust_bma_next_panel & disgust_wfa_next_panel & disgust_wma_next_panel).configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0)
save(disgust_panel, "/mnt/c/Users/pablo/Desktop/dissertation-docs/images/disgust_panel_mturk.png")

## Fear - Female of color

In [51]:
fear_bfa_forced = photo_chart(df_emo_answers_forced, 'fear',  0, df_forced_labels, emotion_st='fear', n_mentions=2, photo=True, yoffset=240)
fear_bfa_free = photo_chart(df_emo_answers_free, 'fear',  0, df_free_labels, emotion_st='fear', n_mentions=2)

/tmp/ipykernel_11424/2691402293.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_stack_single_word['emotion'] = df_stack_single_word['emotion'].str.replace('[^a-zA-Z]', '') # remove non-alphabetic characters


In [52]:
path = '../../emo-ranking-next/fear_bipoc_female/mturk/borda_lilucb_ranking_mturk.csv'

fear_bfa_next = next_chart_wrapper(path, emotion='fear', head=10)

In [53]:
fear_bfa_next_panel = (fear_bfa_forced.properties(width=200) | fear_bfa_free.properties(width=200) | fear_bfa_next.properties(width=200)).properties(title="Female of color - Fear")
fear_bfa_next_panel.configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0).configure_title(dy=-12)

alt.HConcatChart(...)

In [54]:
save(fear_bfa_next_panel, "../../emotions_dashboard/data/fear_bfa_next_panel_mturk.png")

## Fear - Male of color

In [55]:
fear_bma_forced = photo_chart(df_emo_answers_forced, 'fear',  2, df_forced_labels, emotion_st='fear', n_mentions=2, photo=True, yoffset=240)
fear_bma_free = photo_chart(df_emo_answers_free, 'fear',  2, df_free_labels, emotion_st='fear', n_mentions=2)

/tmp/ipykernel_11424/2691402293.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_stack_single_word['emotion'] = df_stack_single_word['emotion'].str.replace('[^a-zA-Z]', '') # remove non-alphabetic characters


In [56]:
path = '../../emo-ranking-next/fear_bipoc_male/mturk/borda_lilucb_ranking_mturk.csv'

fear_bma_next = next_chart_wrapper(path, emotion='fear', head=10)

In [57]:
fear_bma_next_panel = (fear_bma_forced.properties(width=200) | fear_bma_free.properties(width=200) | fear_bma_next.properties(width=200)).properties(title="Male of color - Fear")
fear_bma_next_panel.configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0).configure_title(dy=-12)

alt.HConcatChart(...)

In [58]:
save(fear_bma_next_panel, "../../emotions_dashboard/data/fear_bma_next_panel_mturk.png")

## Fear - White female

In [59]:
fear_wfa_forced = photo_chart(df_emo_answers_forced, 'fear',  20, df_forced_labels, emotion_st='fear', n_mentions=2, photo=True)
fear_wfa_free = photo_chart(df_emo_answers_free, 'fear',  20, df_free_labels, emotion_st='fear', n_mentions=2)

/tmp/ipykernel_11424/2691402293.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_stack_single_word['emotion'] = df_stack_single_word['emotion'].str.replace('[^a-zA-Z]', '') # remove non-alphabetic characters


In [60]:
path = '../../emo-ranking-next/fear_white_female/mturk/borda_lilucb_ranking_mturk.csv'

fear_wfa_next = next_chart_wrapper(path, emotion='fear', head=15)

In [61]:
fear_wfa_next_panel = (fear_wfa_forced.properties(width=200) | fear_wfa_free.properties(width=200) | fear_wfa_next.properties(width=200)).properties(title="White female- Fear")
fear_wfa_next_panel.configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0).configure_title(dy=-12)

alt.HConcatChart(...)

In [62]:
save(fear_wfa_next_panel, "../../emotions_dashboard/data/fear_wfa_next_panel_mturk.png")

## Fear - White male

In [63]:
fear_wma_forced = photo_chart(df_emo_answers_forced, 'fear',  6, df_forced_labels, emotion_st='fear', n_mentions=2, photo=True, yoffset=220)
fear_wma_free = photo_chart(df_emo_answers_free, 'fear',  6, df_free_labels, emotion_st='fear', n_mentions=2)

/tmp/ipykernel_11424/2691402293.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_stack_single_word['emotion'] = df_stack_single_word['emotion'].str.replace('[^a-zA-Z]', '') # remove non-alphabetic characters


In [64]:
path = '../../emo-ranking-next/fear_white_male/mturk/borda_lilucb_ranking_mturk.csv'

fear_wma_next = next_chart_wrapper(path, emotion='fear', head=25)

In [65]:
fear_wma_next_panel = (fear_wma_forced.properties(width=200) | fear_wma_free.properties(width=200) | fear_wma_next.properties(width=200)).properties(title="White male - Fear")
fear_wma_next_panel.configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0).configure_title(dy=-12)

alt.HConcatChart(...)

In [66]:
save(fear_wma_next_panel, "../../emotions_dashboard/data/fear_wma_next_panel_mturk.png")

In [67]:
## JOIN PANELS
fear_panel = (fear_bfa_next_panel & fear_bma_next_panel & fear_wfa_next_panel & fear_wma_next_panel).configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0)
save(fear_panel, "/mnt/c/Users/pablo/Desktop/dissertation-docs/images/fear_panel_mturk.png")

## Happiness - Female of color

In [68]:
happiness_bfa_forced = photo_chart(df_emo_answers_forced, 'happiness',  0, df_forced_labels, emotion_st='happiness', n_mentions=2, photo=True)
happiness_bfa_free = photo_chart(df_emo_answers_free, 'happiness',  0, df_free_labels, emotion_st='happiness', n_mentions=2)

/tmp/ipykernel_11424/2691402293.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_stack_single_word['emotion'] = df_stack_single_word['emotion'].str.replace('[^a-zA-Z]', '') # remove non-alphabetic characters


In [69]:
path = '../../emo-ranking-next/happiness_bipoc_female/mturk/borda_lilucb_ranking_mturk.csv'

happiness_bfa_next = next_chart_wrapper(path, emotion='happiness', head=10)

In [70]:
happiness_bfa_next_panel = (happiness_bfa_forced.properties(width=200) | happiness_bfa_free.properties(width=200) | happiness_bfa_next.properties(width=200)).properties(title="Female of color - happiness")
happiness_bfa_next_panel.configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0).configure_title(dy=-12)

alt.HConcatChart(...)

In [71]:
save(happiness_bfa_next_panel, "../../emotions_dashboard/data/happiness_bfa_next_panel_mturk.png")

## Happiness - Male of color

In [72]:
happiness_bma_forced = photo_chart(df_emo_answers_forced, 'happiness',  2, df_forced_labels, emotion_st='happiness', n_mentions=2, photo=True, yoffset=220)
happiness_bma_free = photo_chart(df_emo_answers_free, 'happiness',  2, df_free_labels, emotion_st='happiness', n_mentions=2)

/tmp/ipykernel_11424/2691402293.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_stack_single_word['emotion'] = df_stack_single_word['emotion'].str.replace('[^a-zA-Z]', '') # remove non-alphabetic characters


In [73]:
path = '../../emo-ranking-next/happiness_bipoc_male/mturk/borda_lilucb_ranking_mturk.csv'

happiness_bma_next = next_chart_wrapper(path, emotion='happiness', head=35)

In [74]:
happiness_bma_next_panel = (happiness_bma_forced.properties(width=200) | happiness_bma_free.properties(width=200) | happiness_bma_next.properties(width=200)).properties(title="Male of color - happiness")
happiness_bma_next_panel.configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0).configure_title(dy=-12)

alt.HConcatChart(...)

In [75]:
save(happiness_bma_next_panel, "../../emotions_dashboard/data/happiness_bma_next_panel_mturk.png")

## Happiness - White female

In [76]:
happiness_wfa_forced = photo_chart(df_emo_answers_forced, 'happiness',  12, df_forced_labels, emotion_st='happiness', n_mentions=2, photo=True, yoffset=220)
happiness_wfa_free = photo_chart(df_emo_answers_free, 'happiness',  12, df_free_labels, emotion_st='happiness', n_mentions=2)

/tmp/ipykernel_11424/2691402293.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_stack_single_word['emotion'] = df_stack_single_word['emotion'].str.replace('[^a-zA-Z]', '') # remove non-alphabetic characters


In [77]:
path = '../../emo-ranking-next/happiness_white_female/mturk/borda_lilucb_ranking_mturk.csv'

happiness_wfa_next = next_chart_wrapper(path, emotion='happiness', head=10)

In [78]:
happiness_wfa_next_panel = (happiness_wfa_forced.properties(width=200) | happiness_wfa_free.properties(width=200) | happiness_wfa_next.properties(width=200)).properties(title="White female - happiness")
happiness_wfa_next_panel.configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0).configure_title(dy=-12)

alt.HConcatChart(...)

In [79]:
save(happiness_wfa_next_panel, "../../emotions_dashboard/data/happiness_wfa_next_panel_mturk.png")

## Happiness - White male

In [80]:
happiness_wma_forced = photo_chart(df_emo_answers_forced, 'happiness',  22, df_forced_labels, emotion_st='happiness', n_mentions=2, photo=True, yoffset=220)
happiness_wma_free = photo_chart(df_emo_answers_free, 'happiness',  22, df_free_labels, emotion_st='happiness', n_mentions=2)

path = '../../emo-ranking-next/happiness_white_male/mturk/borda_lilucb_ranking_mturk.csv'

happiness_wma_next = next_chart_wrapper(path, emotion='happiness', head=10)

/tmp/ipykernel_11424/2691402293.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_stack_single_word['emotion'] = df_stack_single_word['emotion'].str.replace('[^a-zA-Z]', '') # remove non-alphabetic characters


In [81]:
happiness_wma_next_panel = (happiness_wma_forced.properties(width=200) | happiness_wma_free.properties(width=200) | happiness_wma_next.properties(width=200)).properties(title="White male - happiness")
happiness_wma_next_panel.configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0).configure_title(dy=-12)

alt.HConcatChart(...)

In [82]:
save(happiness_wma_next_panel, "../../emotions_dashboard/data/happiness_wma_next_panel_mturk.png")

In [83]:
## JOIN PANELS
happiness_panel = (happiness_bfa_next_panel & happiness_bma_next_panel & happiness_wfa_next_panel & happiness_wma_next_panel).configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0)
save(happiness_panel, "/mnt/c/Users/pablo/Desktop/dissertation-docs/images/happiness_panel_mturk.png")

## Sadness - Female of color

In [84]:
sadness_bfa_forced = photo_chart(df_emo_answers_forced, 'sadness',  0, df_forced_labels, emotion_st='sadness', n_mentions=2, photo=True)
sadness_bfa_free = photo_chart(df_emo_answers_free, 'sadness',  0, df_free_labels, emotion_st='sadness', n_mentions=2)

path = '../../emo-ranking-next/sadness_bipoc_female/mturk/borda_lilucb_ranking_mturk.csv'

sadness_bfa_next = next_chart_wrapper(path, emotion='sadness', head=10)

/tmp/ipykernel_11424/2691402293.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_stack_single_word['emotion'] = df_stack_single_word['emotion'].str.replace('[^a-zA-Z]', '') # remove non-alphabetic characters


In [85]:
sadness_bfa_next_panel = (sadness_bfa_forced.properties(width=200) | sadness_bfa_free.properties(width=200) | sadness_bfa_next.properties(width=200)).properties(title="Female of color - sadness")
sadness_bfa_next_panel.configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0).configure_title(dy=-12)

alt.HConcatChart(...)

In [86]:
save(sadness_bfa_next_panel, "../../emotions_dashboard/data/sadness_bfa_next_panel_mturk.png")

## Sadness - Male of color

In [87]:
sadness_bma_forced = photo_chart(df_emo_answers_forced, 'sadness',  2, df_forced_labels, emotion_st='sadness', n_mentions=2, photo=True, yoffset=260)
sadness_bma_free = photo_chart(df_emo_answers_free, 'sadness',  2, df_free_labels, emotion_st='sadness', n_mentions=2)

path = '../../emo-ranking-next/sadness_bipoc_male/mturk/borda_lilucb_ranking_mturk.csv'

sadness_bma_next = next_chart_wrapper(path, emotion='sadness', head=10)

/tmp/ipykernel_11424/2691402293.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_stack_single_word['emotion'] = df_stack_single_word['emotion'].str.replace('[^a-zA-Z]', '') # remove non-alphabetic characters


In [88]:
sadness_bma_next_panel = (sadness_bma_forced.properties(width=200) | sadness_bma_free.properties(width=200) | sadness_bma_next.properties(width=200)).properties(title="Male of color - sadness")
sadness_bma_next_panel.configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0).configure_title(dy=-12)

alt.HConcatChart(...)

In [89]:
save(sadness_bma_next_panel, "../../emotions_dashboard/data/sadness_bma_next_panel_mturk.png")

## Sadness - White female

In [90]:
sadness_wfa_forced = photo_chart(df_emo_answers_forced, 'sadness',  4, df_forced_labels, emotion_st='sadness', n_mentions=2, photo=True)
sadness_wfa_free = photo_chart(df_emo_answers_free, 'sadness',  4, df_free_labels, emotion_st='sadness', n_mentions=2)

path = '../../emo-ranking-next/sadness_white_female/mturk/borda_lilucb_ranking_mturk.csv'

sadness_wfa_next = next_chart_wrapper(path, emotion='sadness', head=10)

/tmp/ipykernel_11424/2691402293.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_stack_single_word['emotion'] = df_stack_single_word['emotion'].str.replace('[^a-zA-Z]', '') # remove non-alphabetic characters


In [91]:
sadness_wfa_next_panel = (sadness_wfa_forced.properties(width=200) | sadness_wfa_free.properties(width=200) | sadness_wfa_next.properties(width=200)).properties(title="White female - sadness")
sadness_wfa_next_panel.configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0).configure_title(dy=-12)

alt.HConcatChart(...)

In [92]:
save(sadness_wfa_next_panel, "../../emotions_dashboard/data/sadness_wfa_next_panel_mturk.png")

## Sadness - White male

In [93]:
sadness_wma_forced = photo_chart(df_emo_answers_forced, 'sadness',  6, df_forced_labels, emotion_st='sadness', n_mentions=2, photo=True, yoffset=260)
sadness_wma_free = photo_chart(df_emo_answers_free, 'sadness',  6, df_free_labels, emotion_st='sadness', n_mentions=2)

path = '../../emo-ranking-next/sadness_white_male/mturk/borda_lilucb_ranking_mturk.csv'

sadness_wma_next = next_chart_wrapper(path, emotion='sadness', head=15)

/tmp/ipykernel_11424/2691402293.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_stack_single_word['emotion'] = df_stack_single_word['emotion'].str.replace('[^a-zA-Z]', '') # remove non-alphabetic characters


In [94]:
sadness_wma_next_panel = (sadness_wma_forced.properties(width=200) | sadness_wma_free.properties(width=200) | sadness_wma_next.properties(width=200)).properties(title="White male - sadness")
sadness_wma_next_panel.configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0).configure_title(dy=-12)

alt.HConcatChart(...)

In [95]:
save(sadness_wma_next_panel, "../../emotions_dashboard/data/sadness_wma_next_panel_mturk.png")

In [96]:
## JOIN PANELS
sadness_panel = (sadness_bfa_next_panel & sadness_bma_next_panel & sadness_wfa_next_panel & sadness_wma_next_panel).configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0)
save(sadness_panel, "/mnt/c/Users/pablo/Desktop/dissertation-docs/images/sadness_panel_mturk.png")

## Surprise - Female of color

In [97]:
surprise_bfa_forced = photo_chart(df_emo_answers_forced, 'surprise',  0, df_forced_labels, emotion_st='surprise', n_mentions=2, photo=True)
surprise_bfa_free = photo_chart(df_emo_answers_free, 'surprise',  0, df_free_labels, emotion_st='surprise', n_mentions=2)

path = '../../emo-ranking-next/surprise_bipoc_female/mturk/borda_lilucb_ranking_mturk.csv'

surprise_bfa_next = next_chart_wrapper(path, emotion='surprise', head=10)

/tmp/ipykernel_11424/2691402293.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_stack_single_word['emotion'] = df_stack_single_word['emotion'].str.replace('[^a-zA-Z]', '') # remove non-alphabetic characters


In [98]:
surprise_bfa_next_panel = (surprise_bfa_forced.properties(width=200) | surprise_bfa_free.properties(width=200) | surprise_bfa_next.properties(width=200)).properties(title="Female of color - surprise")
surprise_bfa_next_panel.configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0).configure_title(dy=-12)

alt.HConcatChart(...)

In [99]:
save(surprise_bfa_next_panel, "../../emotions_dashboard/data/surprise_bfa_next_panel_mturk.png")

## Surprise - Male of color

In [100]:
surprise_bma_forced = photo_chart(df_emo_answers_forced, 'surprise',  10, df_forced_labels, emotion_st='surprise', n_mentions=2, photo=True, yoffset=260)
surprise_bma_free = photo_chart(df_emo_answers_free, 'surprise',  10, df_free_labels, emotion_st='surprise', n_mentions=2)

path = '../../emo-ranking-next/surprise_bipoc_male/mturk/borda_lilucb_ranking_mturk.csv'

surprise_bma_next = next_chart_wrapper(path, emotion='surprise', head=30)

/tmp/ipykernel_11424/2691402293.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_stack_single_word['emotion'] = df_stack_single_word['emotion'].str.replace('[^a-zA-Z]', '') # remove non-alphabetic characters


In [101]:
surprise_bma_next_panel = (surprise_bma_forced.properties(width=200) | surprise_bma_free.properties(width=200) | surprise_bma_next.properties(width=200)).properties(title="Male of color - surprise")
surprise_bma_next_panel.configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0).configure_title(dy=-12)

alt.HConcatChart(...)

In [102]:
save(surprise_bma_next_panel, "../../emotions_dashboard/data/surprise_bma_next_panel_mturk.png")

## Surprise - White female

In [103]:
surprise_wfa_forced = photo_chart(df_emo_answers_forced, 'surprise',  20, df_forced_labels, emotion_st='surprise', n_mentions=2, photo=True, yoffset=260)
surprise_wfa_free = photo_chart(df_emo_answers_free, 'surprise',  20, df_free_labels, emotion_st='surprise', n_mentions=2)

path = '../../emo-ranking-next/surprise_white_female/mturk/borda_lilucb_ranking_mturk.csv'

surprise_wfa_next = next_chart_wrapper(path, emotion='surprise', head=10)

/tmp/ipykernel_11424/2691402293.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_stack_single_word['emotion'] = df_stack_single_word['emotion'].str.replace('[^a-zA-Z]', '') # remove non-alphabetic characters


In [104]:
surprise_wfa_next_panel = (surprise_wfa_forced.properties(width=200) | surprise_wfa_free.properties(width=200) | surprise_wfa_next.properties(width=200)).properties(title="White female - surprise")
surprise_wfa_next_panel.configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0).configure_title(dy=-12)

alt.HConcatChart(...)

In [105]:
save(surprise_wfa_next_panel, "../../emotions_dashboard/data/surprise_wfa_next_panel_mturk.png")

## Surprise - White male

In [106]:
surprise_wma_forced = photo_chart(df_emo_answers_forced, 'surprise',  22, df_forced_labels, emotion_st='surprise', n_mentions=2, photo=True, yoffset=260)
surprise_wma_free = photo_chart(df_emo_answers_free, 'surprise',  22, df_free_labels, emotion_st='surprise', n_mentions=2)

path = '../../emo-ranking-next/surprise_white_male/mturk/borda_lilucb_ranking_mturk.csv'

surprise_wma_next = next_chart_wrapper(path, emotion='surprise', head=10)

/tmp/ipykernel_11424/2691402293.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_stack_single_word['emotion'] = df_stack_single_word['emotion'].str.replace('[^a-zA-Z]', '') # remove non-alphabetic characters


In [107]:
surprise_wma_next_panel = (surprise_wma_forced.properties(width=200) | surprise_wma_free.properties(width=200) | surprise_wma_next.properties(width=200)).properties(title="White male - surprise")
surprise_wma_next_panel.configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0).configure_title(dy=-12)

alt.HConcatChart(...)

In [108]:
save(surprise_wma_next_panel, "../../emotions_dashboard/data/surprise_wma_next_panel_mturk.png")

In [109]:
## JOIN PANELS
surprise_panel = (surprise_bfa_next_panel & surprise_bma_next_panel & surprise_wfa_next_panel & surprise_wma_next_panel).configure_axis(grid=False, labelFontSize=12, titleFontSize=12).configure_view(strokeWidth=0)
save(surprise_panel, "/mnt/c/Users/pablo/Desktop/dissertation-docs/images/surprise_panel_mturk.png")